In [1]:
import pandas as pd
import numpy as np
import scipy as sp


In [2]:
df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/price_elasticity_YTD.pickle')

In [3]:
df.describe()

,delivery_weeks,item_price,sales
count,3.044708e+07,3.044708e+07,3.044708e+07
mean,3.326022e+00,3.711034e+02,7.267915e-02
std,4.625920e+00,4.814437e+02,5.059961e-01
min,-1.900000e+01,0.000000e+00,0.000000e+00
25%,0.000000e+00,5.999000e+01,0.000000e+00
50%,1.000000e+00,1.651300e+02,0.000000e+00
75%,5.000000e+00,4.999900e+02,0.000000e+00
max,2.010000e+02,7.599990e+03,1.210000e+02


In [ ]:
df_item_code = df[df['item_code']== '000000001000000006']

In [ ]:
df_item_code.describe()